This problem was asked by Uber.

A rule looks like this:

```
A NE B
```

This means this means point A is located northeast of point B.

```
A SW C
```

means that point A is southwest of C.

Given a list of rules, check if the sum of the rules validate. For example:

```
A N B
B NE C
C N A
```

does not validate, since A cannot be both north and south of C.

```
A NW B
A N B
```

is considered valid.

In [2]:
from typing import List

In [11]:
# mapping of nodes to lists of lambda functions representing their relationships to other nodes
node_relationships = {}

# class Rule:
#     def __init__(self, rule_str: str):
#         self.relationships = []

relationship_lambdas = {'N': lambda x0, y0, x1, y1: (x0 == x1 and y0 > y1),
                        'S': lambda x0, y0, x1, y1: (x0 == x1 and y0 < y1),
                        'E': lambda x0, y0, x1, y1: (x0 > x1 and y0 == y1),
                        'W': lambda x0, y0, x1, y1: (x0 < x1 and y0 == y1),
                        'NE': lambda x0, y0, x1, y1: (x0 - x1 == y0 - y1 and x0 > x1),
                        'SW': lambda x0, y0, x1, y1: (x0 - x1 == y0 - y1 and x0 < x1),
                        'SE': lambda x0, y0, x1, y1: (x0 - x1 == y1 - y0 and x0 > x1),
                        'NW': lambda x0, y0, x1, y1: (x0 - x1 == y1 - y0 and x0 < x1)}

opposites = {'N':'S', 'S':'N', 'E':'W', 'W':'E', 'NE':'SW', 'SW':'NE', 'NW':'SE', 'SE':'NW'}
    
def parse_rule(rule: str) -> bool:
    nodeA, rel, nodeB = rule.split(' ')
    
    if nodeA not in node_relationships:
        node_relationships[nodeA] = {}
    if nodeB not in node_relationships:
        node_relationships[nodeB] = {}
        
    if nodeB not in node_relationships[nodeA]:
        node_relationships[nodeA][nodeB] = rel
        node_relationships[nodeB][nodeA] = opposites[rel]
        
    return node_relationships[nodeA][nodeB] == rel
       

def validate_rules(rules: List[str]) -> bool:
    pass
        

In [12]:
assert(validate_rules(['A N B', 'B NE C', 'C N A']) == False)

AssertionError: 

In [6]:
assert(validate_rules(['A NW B', 'A N B']) == True)

AssertionError: 

From here https://medium.com/algorithm-and-datastructure/direction-and-position-rule-verification-445d47641514

> So, we can see that two rules invalidate each other if they relate the same pair of points and are in opposite directions. However, two rules do not invalidate each other if they are in the same or direction or are orthogonal to each other:

> `A N B`<br>
> `A E B`

So the way I have it implemented above is wrong.  `A N B` doesn't mean **strictly** north of B.

And this:

> To add diagonal relationships, we simply parse the two directions into single directions, and treat them as two separate rules.

```
class Solution {
  public static void main(String[] args) {
    test1();
    test2();
    test3();
  }
  private static void test1() {
    String[] rules = {"A N B",
                      "C SE B",
                      "C N A"};
    System.out.println(validate(rules));
  }
  private static void test2() {
    String[] rules = {"A NW B",
                      "A N B"};
    System.out.println(validate(rules));
  }
  private static void test3() {
    String[] rules = {"A N B",
                      "C N B"};
    System.out.println(validate(rules));
  }
  static class Node {
    List<Set<Node>> edges = new ArrayList<>();
    char val;
    public Node(char val) {
      this.val = val;
      for (int i = 0; i < 4; i++)
        edges.add(new HashSet<>());
    }
  }
  public static final int N = 0;
  public static final int E = 1;
  public static final int S = 2;
  public static final int W = 3;
  public static final int[] DIRS = {N, E, S, W};
  public static final Map<Character, Integer> charToDir = new HashMap<>();;
  static {
    charToDir.put('N', N);
    charToDir.put('E', E);
    charToDir.put('S', S);
    charToDir.put('W', W);
  }
  public static boolean validate(String[] rules) {
    Map<Character, Node> map = new HashMap<>();
    for (String line : rules) {
      String[] rule = line.split(" ");
      System.out.println("Rule " + rule[0] + " " + rule[1] + " " + rule[2]);
      char fromVal = rule[2].charAt(0);
      char toVal = rule[0].charAt(0);
      if (!map.containsKey(fromVal)) {
        Node n = new Node(fromVal);
        map.put(fromVal, n);
      }
      if (!map.containsKey(toVal)) {
        Node n = new Node(toVal);
        map.put(toVal, n);
      }
      Node from = map.get(fromVal);
      Node to = map.get(toVal);
      /* Decompose diagonal (two-char) directions to single directions */
      for (char dirChar : rule[1].toCharArray()) {
        int dir = charToDir.get(dirChar);
        if (!isValid(map, from, to, dir))
          return false;
        addEdges(map, from, to, dir);
        System.out.println(from.edges.get(dir));
         System.out.println(to.edges.get(opposite(dir)));
      }
    }
    return true;
  }
  private static int opposite(int dir) {
    return (dir + 2) % 4;
  }
  private static boolean isValid(Map<Character, Node> map,
                                 Node from,
                                 Node to,
                                 int newDir) {
    int oppositeDir = opposite(newDir);
    if (from.edges.get(oppositeDir).contains(to))
          return false;
    return true;
  }
  private static void addEdges(Map<Character, Node> map,
                                  Node from,
                                  Node to,
                                  int newDir) {
    /* Get the direct opposite direction, e.g. S from N */
    int oppositeDir = opposite(newDir);
    /* Add the immediate edge between the nodes, using bi-directional edges. */
    from.edges.get(newDir).add(to);
    to.edges.get(oppositeDir).add(from);
    for (int dir : DIRS) {
      /* Relationships in the same direction are ambiguous.
         For example, if A is north of B, and we are adding 
         C north of B, we cannot say C is north of A. */
      if (dir == newDir)
        continue;
      for (Node neighbor : from.edges.get(dir)) {
        /* No need to add self-edges */
        if (neighbor == to)
          continue;
        /* Add bi-directional edges */
        neighbor.edges.get(newDir).add(to);
        to.edges.get(oppositeDir).add(neighbor);
      }
    }
  }
}
```

> Relationships in the same direction are ambiguous.
         For example, if A is north of B, and we are adding 
         C north of B, we cannot say C is north of A.

The key here is that all of the transitivity between nodes is built up at each step, so all of the pairwise relationships between nodes are always represented in the graph.